## Testing a model for a Kaggle challenge

In [10]:
import pandas as pd
import lightgbm as lgb


df = pd.read_csv('./train.csv')

features = ['f0', 'f1', 'f2', 'region']

X = df[features]
y = df['product']

lgbm_reg = lgb.LGBMRegressor(random_state=42,
                             n_estimators=250,
                             learning_rate=0.05,
                             num_leaves=32,
                             max_depth=-1,
                             reg_alpha=0.1,
                             reg_lambda=0.1,
                             force_col_wise=True)

lgbm_reg.fit(X, y)

test = pd.read_csv('./test.csv')
test['product'] = lgbm_reg.predict(test[features])
test[['id', 'product']].to_csv('answer.csv', index=False)

[LightGBM] [Info] Total Bins 768
[LightGBM] [Info] Number of data points in the train set: 240000, number of used features: 4
[LightGBM] [Info] Start training from score 85.475452


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb # Certifique-se de que esta biblioteca está instalada
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar o dataset
file_path = './train.csv' # Use o nome correto do seu arquivo
try:
    df = pd.read_csv(file_path)

    # --- Engenharia e Seleção de Features ---
    if 'product' not in df.columns:
        print("Erro: Coluna 'product' não encontrada.")
        raise ValueError("Coluna alvo 'product' não encontrada.")

    if df['product'].dtype != 'float64' and df['product'].dtype != 'int64':
        print(f"Aviso: A coluna 'product' é do tipo {df['product'].dtype}. Para regressão, deve ser numérica.")
        df['product'] = pd.to_numeric(df['product'], errors='coerce')
        if df['product'].isnull().any():
            print("Aviso: Valores de 'product' que não puderam ser convertidos para numérico (NaN) foram removidos.")
            df.dropna(subset=['product'], inplace=True)
            if df.empty:
                raise ValueError("O DataFrame ficou vazio após remover valores NaN da coluna 'product'.")

    features = ['f0', 'f1', 'f2', 'region']
    missing_features = [col for col in features if col not in df.columns]
    if missing_features:
        print(f"Erro: As seguintes colunas de features estão faltando: {missing_features}")
        raise ValueError(f"Colunas de features faltando: {missing_features}")

    X = df[features]
    y = df['product']

    if X.empty or y.empty:
         raise ValueError("Features (X) ou alvo (y) estão vazios antes da divisão.")

    # --- Divisão dos Dados ---
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"Formato de X_train: {X_train.shape}")
    print(f"Formato de X_test: {X_test.shape}")

    if X_train.empty or y_train.empty:
        raise ValueError("Dados de treino (X_train ou y_train) estão vazios.")

    # --- Ajuste de Hiperparâmetros com GridSearchCV ---
    print("\nIniciando o ajuste de hiperparâmetros para LGBMRegressor com GridSearchCV...")

    # Define o estimador
    estimator = lgb.LGBMRegressor(random_state=42)

    # Define a grade de parâmetros para testar
    # Esta é uma grade pequena para demonstração. Você pode expandi-la.
    param_grid = {
        'n_estimators': [260, 270, 280],
        'learning_rate': [0.04, 0.05, 0.06],
        'num_leaves': [30, 31, 32], # Padrão é 31
        'max_depth': [-1, 10, 20], # -1 significa sem limite
        'reg_alpha': [0, 0.1, 0.5],
        'reg_lambda': [0, 0.1, 0.5]
    }

    # Configura o GridSearchCV
    # 'neg_mean_squared_error' é usado porque GridSearchCV tenta maximizar a pontuação,
    # então usamos o MSE negativo. Outras opções: 'r2'.
    # cv=3 significa validação cruzada de 3 folds. Aumente se tiver poder computacional.
    # n_jobs=-1 usa todos os processadores disponíveis.
    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        scoring='r2',
        cv=3,
        verbose=1, # Mostra o progresso
        n_jobs=-1 # Use -1 para usar todos os processadores, ou 1 se causar problemas
    )

    # Executa a busca em grade
    grid_search.fit(X_train, y_train)

    print("\nAjuste de hiperparâmetros concluído.")
    print(f"Melhores parâmetros encontrados: {grid_search.best_params_}")
    print(f"Melhor pontuação (neg_mean_squared_error) na validação cruzada: {grid_search.best_score_:.4f}")

    # Pega o melhor estimador encontrado pelo GridSearchCV
    best_lgbm_reg = grid_search.best_estimator_

    # --- Predições com o Melhor Modelo ---
    y_pred_best_lgbm = best_lgbm_reg.predict(X_test)

    # --- Avaliação do Melhor Modelo ---
    print("\n--- Avaliação do Melhor LGBMRegressor (após GridSearchCV) ---")
    mse = mean_squared_error(y_test, y_pred_best_lgbm)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred_best_lgbm)
    r2 = r2_score(y_test, y_pred_best_lgbm)

    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-squared (R2): {r2:.4f}")

except FileNotFoundError:
    print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
except ValueError as ve:
    print(f"ValueError: {ve}")
except Exception as e:
    print(f"Um erro inesperado ocorreu: {e}")
    import traceback
    print(traceback.format_exc())

Formato de X_train: (192000, 4)
Formato de X_test: (48000, 4)

Iniciando o ajuste de hiperparâmetros para LGBMRegressor com GridSearchCV...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001876 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 768
[LightGBM] [Info] Number of data points in the train set: 192000, number of used features: 4
[LightGBM] [Info] Start training from score 85.464094

Ajuste de hiperparâmetros concluído.
Melhores parâmetros encontrados: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 250, 'num_leaves': 32, 'reg_alpha': 0.1, 'reg_lambda': 0.1}
Melhor pontuação (neg_mean_squared_error) na validação cruzada: 0.5690

--- Avaliação do Melhor LGBMRegressor (após GridSearchCV) ---
Mean Squared Error (MSE): 934.3355
Root Mean Squared Error (RMSE): 30.566